In [1]:
import mysql.connector
import json

In [2]:
conn = mysql.connector.connect(
        host="34.16.97.93",
        user="root",
        password="123456789",
        database="CompanyInfo"
)

In [3]:
cursor = conn.cursor()

In [4]:
query = "SELECT table_name, column_name, data_type FROM information_schema.columns WHERE table_schema = %s"
query2 = "SELECT table_name, table_rows FROM information_schema.tables WHERE table_schema = %s AND table_name = %s"
query3 = "SELECT * FROM permissions"

In [5]:
table_sensitivities = dict()
table_sensitivities["projects"] = 1.0
table_sensitivities["users"] = 0.9
table_sensitivities["permissions"] = 0.8
table_sensitivities["code_repo"] = 0.7
table_sensitivities["bug_logs"] = 0.6
table_sensitivities["employee"] = 0.5
table_sensitivities["project_tasks"] = 0.4
table_sensitivities["test_cases"] = 0.4

In [6]:
def table_mappings(table_name):
    if table_name == "bug_logs":
        return ["projects"]
    elif table_name == "code_repo":
        return ["projects"]
    elif table_name == "employee":
        return ["project_tasks","users","permissions"]
    elif table_name == "permissions":
        return ["users","employee"]
    elif table_name == "project_tasks":
        return ["projects","employee"]
    elif table_name == "projects":
        return ["code_repo","project_tasks","bug_logs","test_cases"]
    elif table_name == "test_cases":
        return ["projects"]
    else:
        return ["permissions","employee"]

In [7]:
def save_schema_as_json(tables):
    data = {}
    for table in tables:
        cursor.execute(query2,(conn.database,table))
        row = cursor.fetchone()
        data[table] = {}
        data[table]["num_rows"] = row[1]
        data[table]["sensitivity"] = table_sensitivities[table]
        data[table]["columns"] = []
        data[table]["table_mappings"] = table_mappings(table)

    
    cursor.execute(query,(conn.database,))
    rows = cursor.fetchall()
    
    for row in rows:
        data[row[0]]["columns"].append(row[1])
        
    return data

In [8]:
tables = ["bug_logs","code_repo","employee","permissions","project_tasks","projects","test_cases","users"]

In [9]:
data = save_schema_as_json(tables)

In [10]:
with open("schema.json", "w") as json_file:
    json.dump(data, json_file)

In [1]:
def read_permissions_table():
    cursor.execute(query3)
    rows = cursor.fetchall()
    return rows

In [12]:
rows = read_permissions_table()

[(1, 'Administrator', 'users', 1, 1, 1), (2, 'Project Manager', 'users', 0, 0, 0), (3, 'Team Lead', 'users', 0, 0, 0), (4, 'Developer', 'users', 0, 0, 0), (5, 'Quality Assurance', 'users', 0, 0, 0), (6, 'Quality Assurance', 'employee', 0, 0, 0), (7, 'Developer', 'employee', 0, 0, 0), (8, 'Team Lead', 'employee', 1, 1, 0), (9, 'Project Manager', 'employee', 1, 0, 0), (10, 'Administrator', 'employee', 1, 1, 1), (11, 'Administrator', 'code_repo', 1, 1, 1), (12, 'Project Manager', 'code_repo', 1, 0, 0), (13, 'Team Lead', 'code_repo', 1, 1, 1), (14, 'Developer', 'code_repo', 1, 1, 1), (15, 'Quality Assurance', 'code_repo', 1, 0, 0), (16, 'Administrator', 'projects', 1, 1, 1), (17, 'Project Manager', 'projects', 1, 1, 0), (18, 'Team Lead', 'projects', 1, 0, 0), (19, 'Developer', 'projects', 1, 0, 0), (20, 'Quality Assurance', 'projects', 1, 0, 0), (21, 'Quality Assurance', 'bug_logs', 1, 1, 1), (22, 'Developer', 'bug_logs', 1, 1, 0), (23, 'Team Lead', 'bug_logs', 1, 1, 0), (24, 'Project Mana

In [13]:
permissions = {}
for role in ["Administrator","Project Manager","Team Lead","Developer","Quality Assurance"]:
    permissions[role] = {}

In [14]:
for row in rows:
    permissions[row[1]][row[2]] = [row[3],row[4],row[5]]

In [15]:
with open("permissions.json", "w") as json_file:
    json.dump(permissions, json_file)